In [ ]:
import geopandas as gpd
import plotly.express as px

In [ ]:
if "snakemake" in locals():
    input_path = snakemake.input[0]
    output_path = snakemake.output[0]

else:
    input_path = "../../../results/brussels/trips/passenger_trips_seed1000.gpkg"
    output_path = "../../../results/brussels/trips/passenger_trips_seed1000.xlsx"

In [ ]:
# Read generated data
df = gpd.read_file(input_path).to_crs("EPSG:4326")

In [ ]:
# Location
df["origin_longitude"] = df["geometry"].apply(lambda p: p.coords[0][0])
df["origin_latitude"] = df["geometry"].apply(lambda p: p.coords[0][1])

df["destination_longitude"] = df["geometry"].apply(lambda p: p.coords[1][0])
df["destination_latitude"] = df["geometry"].apply(lambda p: p.coords[1][1])

In [ ]:
# Gender
df["sex"] = df["sex"].replace({ 1: "m", 2: "f" })

# Departure hour
df["departure_time"] = df["reference_time"]

In [ ]:
# Cleanup
df = df[[
    "origin_longitude", "origin_latitude",
    "destination_longitude", "destination_latitude",
    "departure_time", "province", "postcode",
    "age_class", "sex", "cabin_class", "group_size", "luggage_size",
    "frequent_flyer", "purpose", "use_parking", "is_access",
]]

In [ ]:
# Output
df.to_excel(output_path, index = False)

In [ ]:
for attribute in ["departure_time", "age_class", "sex", "cabin_class", "group_size", "luggage_size", "frequent_flyer", "purpose", "is_access", "province", "postcode", "use_parking"]:
    df_plot = df.groupby(attribute).size().reset_index(name = "weight")
    
    if attribute == "postcode":
        print(df_plot)

    else:
        if attribute == "departure_time":
            df_plot["departure_time"] = df_plot["departure_time"] // 3600
        
        figure = px.bar(df_plot, x = attribute, y = "weight", title = attribute)
        figure.show()